In [1]:
# Importing the required libraries, ignoring warnings, setting proper display options, loading the dataset

In [121]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

deliveries = pd.read_csv('IPL_Ball_by_Ball.csv')
matches = pd.read_csv('IPL_Matches.csv')

In [122]:
# Important Terminologies and parameters
# Matches vs Innings
# Matches - No. of times announced in Playing XI
# Innings - No. of times batted

# Batting_Average = Total Runs / Dismissals
# Batting SR = (Total Runs / Balls Faced) * 100
# RPI = Total Runs / Innings

# RPI better than Batting_Average

# Balls per boundary = Balls faced / (Sixes + Fours)
# Dot Balls % = Dots / Balls_faced
# Boundary % = Boundaries / Balls_faced
# Non-boundary % = (Dots + 1s + 2s +3s)/ Balls_faced

In [123]:
# Creating a copy of the datasets
df = deliveries.copy()
mdf = matches.copy()

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ID                260920 non-null  int64 
 1   Innings           260920 non-null  int64 
 2   Overs             260920 non-null  int64 
 3   BallNumber        260920 non-null  int64 
 4   Batter            260920 non-null  object
 5   Bowler            260920 non-null  object
 6   NonStriker        260920 non-null  object
 7   ExtraType         14125 non-null   object
 8   BatsmanRun        260920 non-null  int64 
 9   ExtrasRun         260920 non-null  int64 
 10  TotalRun          260920 non-null  int64 
 11  IsWicketDelivery  260920 non-null  int64 
 12  PlayerOut         12950 non-null   object
 13  Kind              12950 non-null   object
 14  FieldersInvolved  9354 non-null    object
 15  BattingTeam       260920 non-null  object
dtypes: int64(8), object(8)
memory usage: 3

In [129]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               1095 non-null   int64 
 1   City             1044 non-null   object
 2   Date             1095 non-null   object
 3   Season           1095 non-null   object
 4   MatchNumber      1095 non-null   object
 5   Team1            1095 non-null   object
 6   Team2            1095 non-null   object
 7   Venue            1095 non-null   object
 8   TossWinner       1095 non-null   object
 9   TossDecision     1095 non-null   object
 10  WinningTeam      1095 non-null   object
 11  Margin           1095 non-null   int64 
 12  WonBy            1095 non-null   object
 13  Unnamed: 13      70 non-null     object
 14  Player_of_Match  1090 non-null   object
 15  Team1Players     1024 non-null   object
 16  Team2Players     1024 non-null   object
 17  Umpire1          1095 non-null   

In [131]:
df.head()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam
0,335982,1,0,1,SC Ganguly,P Kumar,BB McCullum,legbyes,0,1,1,0,NaN,NaN,NaN,Kolkata Knight Riders
1,335982,1,0,2,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders
2,335982,1,0,3,BB McCullum,P Kumar,SC Ganguly,wides,0,1,1,0,NaN,NaN,NaN,Kolkata Knight Riders
3,335982,1,0,4,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders
4,335982,1,0,5,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders


In [133]:
mdf.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Unnamed: 13,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,335982,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,NaN,['BB McCullum'],"['R Dravid', 'W Jaffer', 'V Kohli', 'JH Kallis', 'CL White', 'MV Boucher', 'B Akhil', 'AA Noffke', 'P Kumar', 'Z Khan', 'SB Joshi']","['SC Ganguly', 'BB McCullum', 'RT Ponting', 'DJ Hussey', 'Mohammad Hafeez', 'LR Shukla', 'WP Saha', 'AB Agarkar', 'AB Dinda', 'M Kartik', 'I Sharma']",Asad Rauf,RE Koertzen
1,335983,Chandigarh,19/04/2008,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,Chennai Super Kings,33,runs,NaN,['MEK Hussey'],"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvraj Singh', 'SM Katich', 'IK Pathan', 'P Dharmani', 'B Lee', 'PP Chawla', 'WA Mota', 'S Sreesanth']","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dhoni', 'SK Raina', 'JDP Oram', 'S Badrinath', 'Joginder Sharma', 'P Amarnath', 'MS Gony', 'M Muralitharan']",MR Benson,SL Shastri
2,335984,Delhi,19/04/2008,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,Delhi Daredevils,9,wickets,NaN,['MF Maharoof'],"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwary', 'KD Karthik', 'R Bhatia', 'M Manhas', 'DL Vettori', 'MF Maharoof', 'B Geeves', 'GD McGrath']","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif', 'DS Lehmann', 'RA Jadeja', 'M Rawat', 'D Salunkhe', 'SK Warne', 'SK Trivedi', 'MM Patel']",Aleem Dar,GA Pratapkumar
3,335985,Mumbai,20/04/2008,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,Royal Challengers Bangalore,5,wickets,NaN,['MV Boucher'],"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', 'RV Uthappa', 'PR Shah', 'AM Nayar', 'SM Pollock', 'Harbhajan Singh', 'MA Khote', 'A Nehra', 'DS Kulkarni']","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', 'JH Kallis', 'V Kohli', 'MV Boucher', 'B Akhil', 'P Kumar', 'Z Khan', 'SB Joshi', 'R Vinay Kumar']",SJ Davis,DJ Harper
4,335986,Kolkata,20/04/2008,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,Kolkata Knight Riders,5,wickets,NaN,['DJ Hussey'],"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC Ganguly', 'DJ Hussey', 'Mohammad Hafeez', 'LR Shukla', 'AB Agarkar', 'AB Dinda', 'M Kartik', 'I Sharma']","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxman', 'A Symonds', 'RG Sharma', 'SB Styris', 'AS Yadav', 'SB Bangar', 'WPUJC Vaas', 'RP Singh', 'PP Ojha']",BF Bowden,K Hariharan


In [135]:
mdf.tail()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Unnamed: 13,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
1090,1426307,Hyderabad,19/05/2024,2024,69,Punjab Kings,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal, Hyderabad",Punjab Kings,bat,Sunrisers Hyderabad,4,wickets,Abhishek Sharma,['Abhishek Sharma'],NaN,NaN,Nitin Menon,VK Sharma
1091,1426309,Ahmedabad,21/05/2024,2024,Qualifier 1,Sunrisers Hyderabad,Kolkata Knight Riders,"Narendra Modi Stadium, Ahmedabad",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,MA Starc,['MA Starc'],NaN,NaN,AK Chaudhary,R Pandit
1092,1426310,Ahmedabad,22/05/2024,2024,Eliminator,Royal Challengers Bengaluru,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,Rajasthan Royals,4,wickets,R Ashwin,['R Ashwin'],NaN,NaN,KN Ananthapadmanabhan,MV Saidharshan Kumar
1093,1426311,Chennai,24/05/2024,2024,Qualifier 2,Sunrisers Hyderabad,Rajasthan Royals,"MA Chidambaram Stadium, Chepauk, Chennai",Rajasthan Royals,field,Sunrisers Hyderabad,36,runs,Shahbaz Ahmed,['Shahbaz Ahmed'],NaN,NaN,Nitin Menon,VK Sharma
1094,1426312,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,MA Starc,['MA Starc'],NaN,NaN,J Madanagopal,Nitin Menon


In [137]:
mdf.drop(columns = ['Unnamed: 13','Team1Players', 'Team2Players'], axis = 1, inplace=True)

In [139]:
mdf.tail()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Player_of_Match,Umpire1,Umpire2
1090,1426307,Hyderabad,19/05/2024,2024,69,Punjab Kings,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal, Hyderabad",Punjab Kings,bat,Sunrisers Hyderabad,4,wickets,['Abhishek Sharma'],Nitin Menon,VK Sharma
1091,1426309,Ahmedabad,21/05/2024,2024,Qualifier 1,Sunrisers Hyderabad,Kolkata Knight Riders,"Narendra Modi Stadium, Ahmedabad",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],AK Chaudhary,R Pandit
1092,1426310,Ahmedabad,22/05/2024,2024,Eliminator,Royal Challengers Bengaluru,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,Rajasthan Royals,4,wickets,['R Ashwin'],KN Ananthapadmanabhan,MV Saidharshan Kumar
1093,1426311,Chennai,24/05/2024,2024,Qualifier 2,Sunrisers Hyderabad,Rajasthan Royals,"MA Chidambaram Stadium, Chepauk, Chennai",Rajasthan Royals,field,Sunrisers Hyderabad,36,runs,['Shahbaz Ahmed'],Nitin Menon,VK Sharma
1094,1426312,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon


In [141]:
# Merging df and mdf

In [143]:
comb = pd.merge(df, mdf, on = 'ID', how = 'left')

In [145]:
comb.tail()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Player_of_Match,Umpire1,Umpire2
260915,1426312,2,9,5,SS Iyer,AK Markram,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon
260916,1426312,2,9,6,VR Iyer,AK Markram,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon
260917,1426312,2,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon
260918,1426312,2,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon
260919,1426312,2,10,3,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon


In [147]:
# Defining a function to get the phase from Overs
def get_phase(over_no):
    if over_no < 6:
     return 'Powerplay'
    elif over_no < 16:
     return 'Middle'
    else:
     return 'Death'

In [149]:
# Creating a new column in comb to show phase
comb['phase'] = comb['Overs'].apply(lambda x : get_phase(x))

In [151]:
comb.head()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Player_of_Match,Umpire1,Umpire2,phase
0,335982,1,0,1,SC Ganguly,P Kumar,BB McCullum,legbyes,0,1,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,['BB McCullum'],Asad Rauf,RE Koertzen,Powerplay
1,335982,1,0,2,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,['BB McCullum'],Asad Rauf,RE Koertzen,Powerplay
2,335982,1,0,3,BB McCullum,P Kumar,SC Ganguly,wides,0,1,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,['BB McCullum'],Asad Rauf,RE Koertzen,Powerplay
3,335982,1,0,4,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,['BB McCullum'],Asad Rauf,RE Koertzen,Powerplay
4,335982,1,0,5,BB McCullum,P Kumar,SC Ganguly,NaN,0,0,0,0,NaN,NaN,NaN,Kolkata Knight Riders,Bangalore,18/04/2008,2007/08,1,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Royal Challengers Bangalore,field,Kolkata Knight Riders,140,runs,['BB McCullum'],Asad Rauf,RE Koertzen,Powerplay


In [153]:
# defining a function to calculate balls_per_dismissal
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1

In [155]:
# defining a function to calculate balls_per_boundary
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [157]:
comb['BowlingTeam'] = comb.apply(lambda row : row['Team1'] if row['BattingTeam'] == row['Team2'] else row['Team2'], axis = 1)

In [159]:
comb.tail()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Player_of_Match,Umpire1,Umpire2,phase,BowlingTeam
260915,1426312,2,9,5,SS Iyer,AK Markram,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260916,1426312,2,9,6,VR Iyer,AK Markram,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260917,1426312,2,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260918,1426312,2,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260919,1426312,2,10,3,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad


In [161]:
# Creating a function which accepts the required dataset, venue, phase and opposition and returns the required data set

In [163]:
def byCustom(comb, current_venue, current_phase, current_opposition):

    comb = comb[comb.Venue == current_venue]
    comb = comb[comb.BowlingTeam == current_opposition]
    comb = comb[comb.phase == current_phase]
    comb.reset_index(inplace = True, drop = True)

    comb['isDot'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 0 else 0)
    comb['isOne'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 1 else 0)
    comb['isTwo'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 2 else 0)
    comb['isThree'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 3 else 0)
    comb['isFour'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 4 else 0)
    comb['isSix'] = comb['BatsmanRun'].apply(lambda x : 1 if x == 6 else 0)

    runs = pd.DataFrame(comb.groupby(['Batter'])['BatsmanRun'].sum()).reset_index().rename(columns = {'BatsmanRun':'runs'})

    balls = pd.DataFrame(comb.groupby(['Batter'])['BallNumber'].count()).reset_index().rename(columns = {'BallNumber':'balls_faced'})

    innings = pd.DataFrame(comb.groupby(['Batter'])['ID'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'ID':'innings'})

    dismissals = pd.DataFrame(comb.groupby(['Batter'])['PlayerOut'].count()).reset_index().rename(columns = {'PlayerOut':'dismissals'})

    sixes = pd.DataFrame(comb.groupby(['Batter'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})

    fours = pd.DataFrame(comb.groupby(['Batter'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})

    threes = pd.DataFrame(comb.groupby(['Batter'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'triples'})

    twos = pd.DataFrame(comb.groupby(['Batter'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'doubles'})

    ones = pd.DataFrame(comb.groupby(['Batter'])['isOne'].sum()).reset_index().rename(columns = {'isOne':'singles'})

    dots = pd.DataFrame(comb.groupby(['Batter'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})

    comb = pd.merge(innings, runs, on = 'Batter').merge(balls, on = 'Batter').merge(dismissals, on = 'Batter').merge(dots, on = 'Batter').merge(ones, on = 'Batter').merge(twos, on = 'Batter').merge(threes, on = 'Batter').merge(fours, on = 'Batter').merge(sixes, on = 'Batter')

    comb['RPI'] = comb.apply(lambda x : (x['runs']/x['innings']), axis = 1)
    comb['SR'] = comb.apply(lambda x : (100 * x['runs'] / x['balls_faced']), axis = 1)
    comb['BPD'] = comb.apply(lambda x : balls_per_dismissal(x['balls_faced'], x['dismissals']), axis = 1)
    comb['BPB'] = comb.apply(lambda x : balls_per_boundary(x['balls_faced'], (x['fours'] + x['sixes'])), axis = 1)
    comb['DP'] = comb.apply(lambda x : x['dots']/x['balls_faced'], axis = 1)



    return comb

In [165]:
# Extracting the names of the venues
comb['Venue'].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Maharashtra Cricket Association Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sh

In [167]:
comb['Venue'].replace(['MA Chidambaram Stadium, Chepauk, Chennai',  'MA Chidambaram Stadium, Chepauk'], 'MA Chidambaram Stadium', inplace = True)

In [169]:
comb['Venue'].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium', 'Dr DY Patil Sports Academy', 'Newlands',
       "St George's Park", 'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Maharashtra Cricket Association Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zaye

In [171]:
comb.tail()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,Margin,WonBy,Player_of_Match,Umpire1,Umpire2,phase,BowlingTeam
260915,1426312,2,9,5,SS Iyer,AK Markram,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,MA Chidambaram Stadium,Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260916,1426312,2,9,6,VR Iyer,AK Markram,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,MA Chidambaram Stadium,Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260917,1426312,2,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,MA Chidambaram Stadium,Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260918,1426312,2,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,MA Chidambaram Stadium,Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad
260919,1426312,2,10,3,VR Iyer,Shahbaz Ahmed,SS Iyer,NaN,1,0,1,0,NaN,NaN,NaN,Kolkata Knight Riders,Chennai,26/05/2024,2024,Final,Sunrisers Hyderabad,Kolkata Knight Riders,MA Chidambaram Stadium,Sunrisers Hyderabad,bat,Kolkata Knight Riders,8,wickets,['MA Starc'],J Madanagopal,Nitin Menon,Middle,Sunrisers Hyderabad


In [175]:
# Calling the byCustom function with required arguements

In [199]:
res = byCustom(comb, 'MA Chidambaram Stadium', 'Middle', 'Chennai Super Kings')

In [201]:
# byCustom function returns the required data set containing information of innings, runs, balls_faced, dismissals, dots,
# singles, doubles, triples, fours, sixes, RPI, SR, BPD, BPB, DP for each batsman.

In [203]:
# Weighted scores for DP, RPI, SR, BPD on the basis of Satty Scale Pair Wise Matrix Normalization method are

In [205]:
wt_sr, wt_rpi, wt_bpd, wt_dp = 0.13, 0.27, 0.16, 0.45

In [207]:
# Filtering the result set with batsman having played atleast 3 innings and faced min 30 balls.

In [209]:
res = res[(res['innings'] >= 3) & (res['balls_faced'] >= 30)]

In [211]:
# Further normalization based on Topsis Method

In [213]:
res.sort_values(by = 'innings', ascending = False).head()

,Batter,innings,runs,balls_faced,dismissals,dots,singles,doubles,triples,fours,sixes,RPI,SR,BPD,BPB,DP
173,V Kohli,8,202,172,6,56,78,20,0,12,6,25.25,117.441860,28.666667,9.555556,0.325581
79,KD Karthik,6,81,75,4,31,29,5,0,9,1,13.50,108.000000,18.750000,7.500000,0.413333
7,AB de Villiers,5,106,79,2,24,32,9,2,11,1,21.20,134.177215,39.500000,6.583333,0.303797
41,DA Miller,5,73,62,4,21,27,6,0,7,1,14.60,117.741935,15.500000,7.750000,0.338710
63,IK Pathan,5,43,52,1,26,21,1,0,2,2,8.60,82.692308,52.000000,13.000000,0.500000


In [215]:
# Calculating squares of SR, RPI, BPD, DP
res['calc_sr'] = res['SR'].apply(lambda x : x*x)
res['calc_rpi'] = res['RPI'].apply(lambda x : x*x)
res['calc_bpd'] = res['BPD'].apply(lambda x : x*x)
res['calc_dp'] = res['DP'].apply(lambda x : x*x)

In [217]:
# Calculating the square roots of sum of squares of SR, RPI, BPD, DP and storing them in the respective varaibles.
sq_SR, sq_RPI, sq_BPD, sq_DPP = np.sqrt(res[['calc_sr', 'calc_rpi', 'calc_bpd', 'calc_dp']].sum(axis = 0))

In [219]:
sq_SR, sq_RPI, sq_BPD, sq_DPP

(629.5418965505909, 111.080982720816, 192.29370258943885, 1.9836793020534984)

In [221]:
# Dividing the squares of SR, RPI, BPD,DP by the square roots of the sum of their squares
res['calc_sr'] = res['calc_sr'].apply(lambda x : x/sq_SR)
res['calc_rpi'] = res['calc_rpi'].apply(lambda x : x/sq_RPI)
res['calc_bpd'] = res['calc_bpd'].apply(lambda x : x/sq_BPD)
res['calc_dp'] = res['calc_dp'].apply(lambda x : x/sq_DPP)

In [223]:
# Multiplying the columns obtained in the previous step with their weighted scores from MCDM.
res['calc_sr'] = res['calc_sr'].apply(lambda x : x*wt_sr)
res['calc_rpi'] = res['calc_rpi'].apply(lambda x : x*wt_rpi)
res['calc_bpd'] = res['calc_bpd'].apply(lambda x : x*wt_bpd)
res['calc_dp'] = res['calc_dp'].apply(lambda x : x*wt_dp)

In [225]:
# Calculating their best and worst values
best_sr, worst_sr = max(res['calc_sr']), min(res['calc_sr'])
best_rpi, worst_rpi = max(res['calc_rpi']), min(res['calc_rpi'])
best_bpd, worst_bpd = max(res['calc_bpd']), min(res['calc_bpd'])
best_dp, worst_dp = max(res['calc_dp']), min(res['calc_dp'])

In [227]:
# Calculating the square of the difference between the values obtained and the best values
res['dev_best_SR'] = res['calc_sr'].apply(lambda x : (x-best_sr) * (x-best_sr))
res['dev_best_RPI'] = res['calc_rpi'].apply(lambda x : (x-best_rpi) * (x-best_rpi))
res['dev_best_BPD'] = res['calc_bpd'].apply(lambda x : (x-best_bpd) * (x-best_bpd))
res['dev_best_DP'] = res['calc_dp'].apply(lambda x : (x-best_dp) * (x-best_dp))

In [229]:
# Adding the values obtained in the previous step of SR, RPI, BPD, DP
res['dev_best_sqrt'] = res.apply(lambda x : x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_DP'], axis = 1)

In [231]:
# Calculating the square of the difference between the values obtained and the worst values
res['dev_worst_SR'] = res['calc_sr'].apply(lambda x : (x-worst_sr) * (x-worst_sr))
res['dev_worst_RPI'] = res['calc_rpi'].apply(lambda x : (x-worst_rpi) * (x-worst_rpi))
res['dev_worst_BPD'] = res['calc_bpd'].apply(lambda x : (x-worst_bpd) * (x-worst_bpd))
res['dev_worst_DP'] = res['calc_dp'].apply(lambda x : (x-worst_dp) * (x-worst_dp))

In [233]:
# Adding the values obtained in the previous step of SR, RPI, BPD, DP
res['dev_worst_sqrt'] = res.apply(lambda x : x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_DP'], axis = 1)

In [235]:
# Calculating the required score by dividing dev_worst_sqrt by sum of dev_worst_sqrt and dev_best_sqrt
res['score'] = res.apply(lambda x : x['dev_worst_sqrt']/ (x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [237]:
res.head(5)

,Batter,innings,runs,balls_faced,dismissals,dots,singles,doubles,triples,fours,sixes,RPI,SR,BPD,BPB,DP,calc_sr,calc_rpi,calc_bpd,calc_dp,dev_best_SR,dev_best_RPI,dev_best_BPD,dev_best_DP,dev_best_sqrt,dev_worst_SR,dev_worst_RPI,dev_worst_BPD,dev_worst_DP,dev_worst_sqrt,score
7,AB de Villiers,5,106,79,2,24,32,9,2,11,1,21.20,134.177215,39.500000,6.583333,0.303797,3.717716,1.092435,1.298222,0.020937,12.897642,4.995837,14.167951,0.002108,32.063538,7.207106,0.930596,1.455613,0.000133,9.593448,0.230296
10,AD Russell,4,74,60,1,24,24,3,0,5,4,18.50,123.333333,60.000000,6.666667,0.400000,3.141085,0.831893,2.995418,0.036296,17.371895,6.228415,4.271821,0.000934,27.873064,4.443553,0.495801,8.431376,0.000724,13.371455,0.324200
17,AT Rayudu,4,29,41,2,13,27,1,0,0,0,7.25,70.731707,20.500000,41.000000,0.317073,1.033111,0.127762,0.349673,0.022807,39.387350,10.238789,22.208437,0.001940,71.836516,0.000000,0.000000,0.066532,0.000180,0.066713,0.000928
41,DA Miller,5,73,62,4,21,27,6,0,7,1,14.60,117.741935,15.500000,7.750000,0.338710,2.862734,0.518119,0.199903,0.026025,19.769680,7.893026,23.642484,0.001667,51.306857,3.347522,0.152379,0.011700,0.000277,3.511878,0.064063
42,DA Warner,4,109,100,3,32,47,15,0,2,4,27.25,109.000000,33.333333,16.666667,0.320000,2.453419,1.804916,0.924512,0.023230,23.577107,2.318481,17.120930,0.001903,43.018421,2.017275,2.812848,0.693518,0.000192,5.523832,0.113794


In [239]:
res.reset_index(inplace = True, drop = True)

In [241]:
# Keeping only the required columns

In [243]:
res2 = res[['Batter','innings','runs','balls_faced','dismissals','dots', 'SR', 'BPD', 'RPI', 'score']].sort_values(by = 'score', ascending = False).head(10)

In [245]:
# After completing the same process for opposition as CSK and venue as Chepauk these are the top 10 batsmen

In [249]:
res2.reset_index(inplace = True, drop = True)

In [251]:
res2

,Batter,innings,runs,balls_faced,dismissals,dots,SR,BPD,RPI,score
0,SE Marsh,3,111,59,1,12,188.135593,59.000000,37.000000,0.924514
1,G Gambhir,3,92,78,1,23,117.948718,78.000000,30.666667,0.611935
2,SR Watson,4,124,75,3,19,165.333333,25.000000,31.000000,0.519104
3,AD Russell,4,74,60,1,24,123.333333,60.000000,18.500000,0.324200
4,MS Bisla,3,94,69,2,21,136.231884,34.500000,31.333333,0.317440
5,NV Ojha,3,69,46,2,15,150.000000,23.000000,23.000000,0.307948
6,AB de Villiers,5,106,79,2,24,134.177215,39.500000,21.200000,0.230296
7,Y Venugopal Rao,3,67,52,2,17,128.846154,26.000000,22.333333,0.152057
8,MK Tiwary,3,52,49,1,14,106.122449,49.000000,17.333333,0.121495
9,V Kohli,8,202,172,6,56,117.441860,28.666667,25.250000,0.118309


In [ ]:
# Shaun Marsh is clearly the best batsman to bat in the middle overs vs CSK at Chepauk
# followed by Gautam Gambhir and Shane Watson.
# where as Kohli has been dismissed 6 times vs CSK at Chepauk